In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from pprint import pprint
from citipy import citipy
import requests
import time
import datetime

# weather.io API Key
#from config import api_key

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


595

In [33]:
cities = ["denver","dallas","hobart"]
cities_df = pd.DataFrame(cities)
#cities_df.rename({1:"City"},inplace=True)
cities_df["lat"]=-99
cities_df["temp"]=-99
cities_df["sunrise"]=-99
cities_df["sunset"]=-99
cities_df["solar_rad"]=-99
cities_df["elev_angle"]=-99
cities_df["day_length"]=-99
cities_df.head()

,0,lat,temp,sunrise,sunset,solar_rad,elev_angle,day_length
0,denver,-99,-99,-99,-99,-99,-99,-99
1,dallas,-99,-99,-99,-99,-99,-99,-99
2,hobart,-99,-99,-99,-99,-99,-99,-99


In [36]:
def timeDiff(sunrise, sunset):
# some stations have a sunrise and sunset of 0 because the sun is always above
# or below the horizon
   # if (sunrise > 0 and sunset > 0):
        
#  Convert the integer sunrise and sunset to actual date-time variables
        #rise = datetime.datetime.fromtimestamp(sunrise)
        #set = datetime.datetime.fromtimestamp(sunset)
        rise = sunrise
        sset = sunset
        fmt = '%Y-%m-%d %H:%M:%S'
               
#  This converts to date/time objects
        d1 = datetime.datetime.strptime(rise, fmt)
        d2 = datetime.datetime.strptime(sset, fmt)
        
#  compute the number of minutes between the two dates
        diff = d2 - d1
          
        return ((diff.days * 24 * 60) + (diff.seconds/60))
   # else:
     #   return (0)


In [41]:

for i,city in enumerate(cities):
    url = f"https://api.weatherbit.io/v2.0/current?city={city}&key=70361cbf92284b03bfbfaac344ac49ef"
    print(url)
    city_data = requests.get(url).json()
    try:
        lat = city_data["data"][0]["lat"]
        temp = city_data["data"][0]["temp"]
        solar_rad = city_data["data"][0]["solar_rad"]
        elev_angle = city_data["data"][0]["elev_angle"]
        sunrise = city_data["data"][0]["sunrise"]
        sunset = city_data["data"][0]["sunset"]
        ob_time = city_data["data"][0]["ob_time"]
        
        hold = ob_time.split(" ")
        ob_time = hold[0]
        
        sunrise = f"{ob_time} {sunrise}:00" 
        sunset = f"{ob_time} {sunset}:00"
        
        minutes = timeDiff(sunrise, sunset)
   
        # "ob_time":"2017-08-28 16:45"
        
        cities_df.iloc[i, cities_df.columns.get_loc("lat")] = lat
        cities_df.iloc[i, cities_df.columns.get_loc("temp")] = temp
        cities_df.iloc[i, cities_df.columns.get_loc("sunrise")] = sunrise
        cities_df.iloc[i, cities_df.columns.get_loc("sunset")] = sunset
        cities_df.iloc[i, cities_df.columns.get_loc("day_length")] = minutes
        
    except:
        print("missed " + city)
cities_df.head() 
#cities_df.to_csv('cities.csv', index=False)

https://api.weatherbit.io/v2.0/current?city=denver&key=70361cbf92284b03bfbfaac344ac49ef
https://api.weatherbit.io/v2.0/current?city=dallas&key=70361cbf92284b03bfbfaac344ac49ef
https://api.weatherbit.io/v2.0/current?city=hobart&key=70361cbf92284b03bfbfaac344ac49ef


,0,lat,temp,sunrise,sunset,solar_rad,elev_angle,day_length
0,denver,35.53125,27.0,2020-10-24 11:39:00,2020-10-24 22:37:00,-99,-99,658.0
1,dallas,33.92371,23.9,2020-10-24 11:53:00,2020-10-24 22:54:00,-99,-99,661.0
2,hobart,35.02950,9.0,2020-10-24 12:51:00,2020-10-24 23:50:00,-99,-99,659.0
